<a href="https://colab.research.google.com/github/peshqa/MMO_lab/blob/main/lab7_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Инструменты для работы с языком

... или зачем нужна предобработка.

Раньше мы смотрели на светлую сторону анализа данных - построение моделей. Теперь попробуем глубже посмотреть на часть про предобработку данных. Задача предобработки особенно актуальна, если мы имеем дело с текстами.

## Задача: классификация твитов по тональности

У нас есть выборка из твитов.
Нам известна эмоциональная окраска каждого твита из выборки: положительная или отрицательная. Задача состоит в построении модели, которая по тексту твита предсказывает его эмоциональную окраску.

Классификацию по тональности используют в рекомендательных системах, чтобы понять, понравилось ли людям кафе, кино, etc.
Выборка ниже в этом разделе


In [ ]:
import pandas as pd # библиотека для удобной работы с датафреймами
import numpy as np # библиотека для удобной работы со списками и матрицами

# библиотека, где реализованы основные алгоритмы машинного обучения
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

Откроем файлы и создадим массив из текстов и правильных меток для твитов.
Сначала идут положительные твиты, потом отрицательные.

In [ ]:
# загружаем положительные твиты
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive) # устанавливаем метки

# загружаем отрицательные твиты
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative) # устанавливаем метки

# соединяем вместе
df = positive.append(negative)

FileNotFoundError: ignored

Посмотрим на полученные данные:

In [ ]:
df.sample(5, random_state=40)

,text,label
15931,RT @Blawar_1337: Теперь у нас с @Wake_UA появи...,positive
59532,с днём рождения зайка*))) ухх погуляем мы сего...,positive
47185,RT @Shumkova0406199: @ann_safina Вов вов вов А...,negative
42002,"Надо выдернуть звуковую дорожку из ""Доктора Ка...",positive
109035,@_hassliebe_ может все таки на этой неделе вер...,negative


Разбиваем данные на обучающую и тестовую выборки с помощью функции ```train_test_split()``` из **sklearn**:


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df.text, df.label)

### Векторизаторы

Векторизатор преобразует слово или набор слов в числовой вектор, понятный алгоритму машинного обучения, который привык работать с числовыми табличными данными.

Ниже - пример преобразования слов в двумерных вектор, каждому слову соответствует точка на плоскости.

<a href="https://drive.google.com/uc?id=1ukv-FTj0jeVdcgVlOaNBocUfNuYGGVZg
" target="_blank"><img src="https://drive.google.com/uc?id=1ukv-FTj0jeVdcgVlOaNBocUfNuYGGVZg"
alt="IMAGE ALT TEXT HERE" width="600" border="0" /></a>

На начальном этапе нам будет достаточно тех инструментов, которые уже есть в знакомой нам библиотеке **sklearn**.

In [ ]:
from sklearn.linear_model import LogisticRegression # можно заменить на любимый классификатор
from sklearn.feature_extraction.text import CountVectorizer # модель "мешка слов", см. далее

Самый простой способ извлечь признаки из текстовых данных -- векторизаторы: `CountVectorizer` и `TfidfVectorizer`

Объект `CountVectorizer` делает следующую вещь:
* строит для каждого документа (каждой пришедшей ему строки) вектор размерности `n`, где `n` -- количество слов или n-грам во всём корпусе
* заполняет каждый i-тый элемент количеством вхождений слова в данный документ

<a href="https://drive.google.com/uc?id=1ukv-FTj0jeVdcgVlOaNBocUfNuYGGVZg
" target="_blank"><img src="https://drive.google.com/uc?id=1jHmkrGZTMawM46Yzxh243Ur1y5pYKzrl"
alt="IMAGE ALT TEXT HERE" width="600" border="0" /></a>

Инициализируем ```CountVectorizer()```, указав в качестве признаков униграммы:

In [ ]:
vectorizer = CountVectorizer()

После инициализации _vectorizer_ можно обучить на наших данных.

Для обучения используем обучающую выборку ```x_train```, но в отличие от классификатора мы используем метод ```fit_transform()```: сначала обучаем наш векторизатор, а потом сразу применяем его к нашему набору данных. Это похоже на то, как мы работали с label encoderом и one-hot-encoderом.


In [ ]:
vectorized_x_train = vectorizer.fit_transform(x_train)

Так как результат не зависит от порядка слов в текстах, то говорят, что такая модель представления текстов в виде векторов получается из *гипотезы представления текста как мешка слов*

В vectorizer.vocabulary_ лежит словарь, отображение слов в их индексы:

In [ ]:
list(vectorizer.vocabulary_.items())[:10]

[('rt', 74533),
 ('ktvsktvs', 49068),
 ('влюблен', 113229),
 ('слово', 211036),
 ('бывает', 109197),
 ('ибо', 138812),
 ('оно', 173471),
 ('везде', 110816),
 ('уместно', 227222),
 ('ты', 224529)]

В нашей выборке 170125 текстов (твитов), в них встречается 243591 разных слов.

In [ ]:
vectorized_x_train.shape

(170125, 243591)

Так как теперь у нас есть **численное представление** и набор входных признаков, то мы можем обучить модель логистической регрессии (или любую другую из тех, на которые мы смотрели раньше)

In [ ]:
clf = LogisticRegression(random_state=42, max_iter=1000, C=1) # фиксируем random_state для воспроизводимости результатов
clf.fit(vectorized_x_train, y_train)


LogisticRegression(C=1, max_iter=1000, random_state=42)

С тестовыми данными нужно проделать то же самое, что и с данными для обучения: сделать из текстов вектора, которые можно передавать в классификатор для прогноза класса объекта.

У нас уже есть обученный векторизатор ```vectorizer```, поэтому используем метод ```transform()``` (просто применить его), а не ```fit_transform``` (обучить и применить).

In [ ]:
vectorized_x_test = vectorizer.transform(x_test)
clf.score(vectorized_x_test,y_test)

0.765169549806909

Как раньше, для получения прогноза у обученного классификатора используем метод ```predict()```.

С помощью функции ```classification_report()```, которая считает сразу несколько метрик качества классификации, посмотрим на то, насколько хорошо мы предсказываем положительную или отрицательную тональность твита .

In [ ]:
pred = clf.predict(vectorized_x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

    negative       0.76      0.77      0.76     27920
    positive       0.77      0.76      0.77     28789

    accuracy                           0.77     56709
   macro avg       0.77      0.77      0.77     56709
weighted avg       0.77      0.77      0.77     56709



## Биграммы


Попробуем сделать то же самое, ДОБАВИВ в качестве признаков биграммы:


In [ ]:
# инициализируем векторайзер
trigram_vectorizer = CountVectorizer(ngram_range=(1, 2))

# обучаем его и сразу применяем к x_train
trigram_vectorized_x_train = trigram_vectorizer.fit_transform(x_train)

# инициализируем и обучаем классификатор
clf = LogisticRegression(random_state=42)
clf.fit(trigram_vectorized_x_train, y_train)

# применяем обученный векторизатор к тестовым данным
trigram_vectorized_x_test = trigram_vectorizer.transform(x_test)

# получаем предсказания и выводим информацию о качестве
pred = clf.predict(trigram_vectorized_x_test)
print(classification_report(y_test, pred))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

    negative       0.77      0.78      0.78     27920
    positive       0.78      0.78      0.78     28789

    accuracy                           0.78     56709
   macro avg       0.78      0.78      0.78     56709
weighted avg       0.78      0.78      0.78     56709



##  TF-IDF векторизация

`TfidfVectorizer` делает то же, что и `CountVectorizer`, но в качестве значений выдает **tf-idf** каждого слова.

Как считается tf-idf:

**TF (term frequency)** – относительная частотность слова в документе:
$$ TF(t,d) = \frac{n_{t}}{\sum_k n_{k}} $$

**IDF (inverse document frequency)** – обратная частота документов, в которых есть это слово:
$$ IDF(t, D) = \mbox{log} \frac{|D|}{|{d : t \in d}|} $$

Перемножаем их:
$$TFIDF(t, d, D) = TF(t,d) \times IDF(i, D)$$

Сакральный смысл: если слово часто встречается в одном документе, но в целом по корпусу встречается в небольшом
количестве документов, у него высокий TF-IDF.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

Действуем аналогично, как с ```CountVectorizer()```:

In [ ]:
# инициализируем векторизатор, в качестве переменных используем униграммы и биграммы
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2))

# обучаем его и сразу применяем к x_train
tfidf_vectorized_x_train = tfidf_vectorizer.fit_transform(x_train)

# инициализируем и обучаем классификатор
clf = LogisticRegression(random_state=42)
clf.fit(tfidf_vectorized_x_train, y_train)

# применяем обученный векторизатор к тестовым данным
tfidf_vectorized_x_test = tfidf_vectorizer.transform(x_test)

# получаем предсказания и выводим информацию о качестве
pred = clf.predict(tfidf_vectorized_x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

    negative       0.77      0.75      0.76     27920
    positive       0.76      0.78      0.77     28789

    accuracy                           0.76     56709
   macro avg       0.76      0.76      0.76     56709
weighted avg       0.76      0.76      0.76     56709



## Стоп-слова и пунктуация

**Стоп-слова** - это слова, которые часто встречаются практически в любом тексте и ничего интересного не говорят о конретном документе. Для модели это просто шум. А шум нужно убирать. По аналогичной причине убирают и пунктуацию.

In [ ]:
# импортируем стоп-слова из библиотеки nltk
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
s= stopwords.words("russian")

# посмотрим на стоп-слова для русского языка
print(s)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


*Знаки* пунктуации лучше импортировать из модуля **String**. В нем хранятся различные наборы констант для работы со строками (пунктуация, алфавит и др.).

In [ ]:
from string import punctuation
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

Объединим стоп-слова и знаки пунктуации вместе и запишем в переменную ```noise```:

In [ ]:
noise = stopwords.words('russian') + list(punctuation)

Теперь нужно обучать нашу модель с учетом новых знаний про токенизацию и стоп-слова.

Для этого мы можем собрать новый векторизатор, передав ему на вход:
* какие n-граммы нам нужны, параметр **ngram_range**;
* какой токенизатор мы используем, параметр **tokenizer**;
* какие у нас стоп-слова, параметр **stop_words**.

*Напоминание:* мы используем готовый токенизатор ```word_tokenize```, а стоп-слова хранятся в переменной ```noise```

In [ ]:
# инициализируем умный векторайзер
smart_vectorizer = CountVectorizer(ngram_range=(1, 2),
                                   stop_words=noise)

In [ ]:
# обучаем его и сразу применяем к x_train
smart_vectorized_x_train = smart_vectorizer.fit_transform(x_train)

# инициализируем и обучаем классификатор
clf = LogisticRegression(random_state=42)
clf.fit(smart_vectorized_x_train, y_train)

# применяем обученный векторайзер к тестовым данным
smart_vectorized_x_test = smart_vectorizer.transform(x_test)

# получаем предсказания и выводим информацию о качестве
pred = clf.predict(smart_vectorized_x_test)
print(classification_report(y_test, pred))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

    negative       0.76      0.77      0.76     27920
    positive       0.77      0.76      0.77     28789

    accuracy                           0.76     56709
   macro avg       0.76      0.76      0.76     56709
weighted avg       0.76      0.76      0.76     56709





Что ещё можно сделать?

## Лемматизация

**Лемматизация** – это сведение разных форм одного слова к начальной форме – **лемме**. Почему это хорошо?
* Во-первых, естественно рассматривать как отдельный признак каждое *слово*, а не каждую его отдельную форму.
* Во-вторых, некоторые стоп-слова стоят только в начальной форме, и без лематизации выкидываем мы только её.

Для русского есть хороших лемматизатор pymorphy.

### [Pymorphy](http://pymorphy2.readthedocs.io/en/latest/)
Это модуль на питоне, довольно быстрый и с кучей функций.

In [ ]:
# устанавливаем pymorphy2
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 82.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13707 sha256=f50f25d3d8991f69d4d65c089fe681806a30fb2cb887ba6641056a4984203d65
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


В pymorphy2 для морфологического анализа слов есть ```MorphAnalyzer()```:

In [ ]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [ ]:
def parse(x):
    return [morph.parse(w)[0].normal_form for w in x]

In [ ]:
df.text = df.text.apply(parse)
df.sample(5, random_state=40)

In [ ]:
#здесь надо дописать классификацию с учетом лемматизации

## О важности эксплоративного анализа

Но иногда пунктуация бывает и не шумом - главное отталкиваться от задачи. Что будет если вообще не убирать пунктуацию?

In [ ]:
#здесь надо подключть nltk и word_tokenize
import nltk
nltk.download('punkt')
#df['message'] = df['message'].apply(nltk.word_tokenize)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# инициализируем умный векторайзер stop-words НЕ ИСПОЛЬЗУЕМ!
alternative_tfidf_vectorizer = TfidfVectorizer(tokenizer=nltk.word_tokenize)

# обучаем его и сразу применяем к x_train
alternative_tfidf_vectorized_x_train = alternative_tfidf_vectorizer.fit_transform(x_train)

# инициализируем и обучаем классификатор
clf = LogisticRegression(random_state=42)
clf.fit(alternative_tfidf_vectorized_x_train, y_train)

# применяем обученный векторайзер к тестовым данным
alternative_tfidf_vectorized_x_test = alternative_tfidf_vectorizer.transform(x_test)

# получаем предсказания и выводим информацию о качестве
pred = clf.predict(alternative_tfidf_vectorized_x_test)
print(classification_report(y_test, pred))

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


              precision    recall  f1-score   support

    negative       1.00      1.00      1.00     27920
    positive       1.00      1.00      1.00     28789

    accuracy                           1.00     56709
   macro avg       1.00      1.00      1.00     56709
weighted avg       1.00      1.00      1.00     56709



Шок! Стоило оставить пунктуацию -- и все метрики равны 1. Как это получилось? Среди неё были очень значимые токены (как вы думаете, какие?).

Посмотрим, как один из супер-значительных токенов справится с классификацией безо всякого машинного обучения:

In [ ]:
cool_token = ')'
pred = ['positive' if cool_token in tweet else 'negative' for tweet in x_test]
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       1.00      0.85      0.92     32834
    positive       0.83      1.00      0.91     23875

    accuracy                           0.91     56709
   macro avg       0.91      0.93      0.91     56709
weighted avg       0.93      0.91      0.91     56709



## Символьные n-граммы

Теперь в качестве фичей используем, например, униграммы символов. Для этого необходимо установить в ```CountVectorizer()``` параметр ```analyzer = 'char'```, то есть анализировать символы.

In [ ]:
# инициализируем векторайзер для символов
char_vectorizer = CountVectorizer(analyzer='char')

# обучаем его и сразу применяем к x_train
char_vectorized_x_train = char_vectorizer.fit_transform(x_train)

# инициализируем и обучаем классификатор
clf = LogisticRegression(random_state=42)
clf.fit(char_vectorized_x_train, y_train)

# применяем обученный векторайзер к тестовым данным
char_vectorized_x_test = char_vectorizer.transform(x_test)

# получаем предсказания и выводим информацию о качестве
pred = clf.predict(char_vectorized_x_test)
print(classification_report(y_test, pred))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

    negative       1.00      0.99      1.00     27920
    positive       0.99      1.00      1.00     28789

    accuracy                           1.00     56709
   macro avg       1.00      1.00      1.00     56709
weighted avg       1.00      1.00      1.00     56709



Из предыдущего раздела уже понятно, почему на этих данных точность равна 1.

Символьные n-граммы используются, например, для задачи определения языка. Ещё одна замечательная особенность признаков-символов - для них не нужна токенизация и лемматизация, можно использовать такой подход для языков, у которых нет готовых анализаторов.